### Import libraries

In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from pymodulon.gene_util import *
import os

### Read in GFF file 

In [2]:
gff_files = '/home/kkrishnan/SBRG/Sequencing/Y-Lipolytica/ICA/97_dimensionality_analysis/Data/COGfiles/GCF_000002525.2_ASM252v1_genomic.gff'

keep_cols = ['accession','start','end','strand','gene_name','old_locus_tag','gene_product','ncbi_protein']

DF_annot = gff2pandas(gff_files,index='locus_tag')
DF_annot = DF_annot[keep_cols]

DF_annot.head()

,accession,start,end,strand,gene_name,old_locus_tag,gene_product,ncbi_protein
locus_tag,,,,,,,,
YALI0_B00110g,NC_006068.1,234.0,764.0,+,None,YALI0B00110g,YALI0B00110p,XP_500334.1
YALI0_E00110g,NC_006071.1,435.0,1553.0,-,None,YALI0E00110g,YALI0E00110p,XP_503359.1
YALI0_B00132g,NC_006068.1,1930.0,3360.0,-,None,YALI0B00132g,YALI0B00132p,XP_500335.1
YALI0_E00132g,NC_006071.1,2299.0,3738.0,-,None,YALI0E00132g,YALI0E00132p,XP_503360.1
YALI0_D00110g,NC_006070.1,3558.0,4058.0,-,None,YALI0D00110g,YALI0D00110p,XP_502226.1


### Extract sequences for each gene from fna file

In [3]:
fasta_files = ['/home/kkrishnan/SBRG/Sequencing/Y-Lipolytica/ICA/97_dimensionality_analysis/Data/COGfiles/GCF_000002525.2_ASM252v1_genomic.fna']

cds_list = []
for fasta in fasta_files:
    seq = list(SeqIO.parse(fasta,'fasta'))
    gene_id = []
    for gene in seq:
        gene_id.append(gene.id)
        
    for curr_id in gene_id:
        # Get gene information for genes in this fasta file
        df_genes = DF_annot[DF_annot.accession == curr_id]

        for i,row in df_genes.iterrows():
            cds = seq[0][int(row.start-1):int(row.end)]
            if row.strand == '-':
                cds = seq[0][int(row.start-1):int(row.end)].reverse_complement()
            cds.id = row.name
            cds.description = row.gene_name if pd.notnull(row.gene_name) else row.name
            cds_list.append(cds)

### Save CDS file for eggnog

In [4]:
#SeqIO.write(cds_list,'CDS.fna','fasta')

In [10]:
eggnog_file = os.getcwd() + '/Data/Annotations/emapper_annotations.tsv'
DF_eggnog = pd.read_csv(eggnog_file,sep='\t')
# eggnog_cols = ['query_name','seed eggNOG ortholog','seed ortholog evalue','seed ortholog score',
#                'Predicted taxonomic group','Predicted protein name','Gene Ontology terms',
#                'EC number','KEGG_orth','KEGG_pathway','KEGG_module','KEGG_reaction',
#                'KEGG_rclass','BRITE','KEGG_TC','CAZy','BiGG Reaction','tax_scope',
#                'eggNOG OGs','bestOG_deprecated','COG','eggNOG free text description']

#DF_eggnog.columns = eggnog_cols

# Strip last three rows as they are comments
DF_eggnog = DF_eggnog.iloc[:-3]

# Set locus tag as index
DF_eggnog = DF_eggnog.set_index('Gene')
DF_eggnog.index.name = 'locus_tag'

DF_eggnog.head()

,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,EC,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
locus_tag,,,,,,,,,,,,,,,,,,,,
YALI0_A00264g,4952.CAG83529,0.000000e+00,2194.0,"COG0060@1|root,KOG0434@2759|Eukaryota,38CIS@33...",4891|Saccharomycetes,J,Belongs to the class-I aminoacyl-tRNA syntheta...,ILS1,"GO:0003674,GO:0003824,GO:0004812,GO:0004822,GO...",6.1.1.5,ko:K01870,"ko00970,map00970","M00359,M00360",R03656,"RC00055,RC00523","ko00000,ko00001,ko00002,ko01000,ko01007,ko03016",-,-,-,"Anticodon_1,tRNA-synt_1"
YALI0_A00286g,4952.CAG83530,3.570000e-260,713.0,"COG1163@1|root,KOG1487@2759|Eukaryota,38DRY@33...",4891|Saccharomycetes,T,to Saccharomyces cerevisiae RBG1 (YAL036C),-,"GO:0000166,GO:0001882,GO:0001883,GO:0002181,GO...",-,ko:K06944,-,-,-,-,ko00000,-,-,-,"MMR_HSR1,MMR_HSR1_Xtn,TGS"
YALI0_A00396g,4952.CAG83535,7.670000e-243,681.0,"28MZB@1|root,2QUI6@2759|Eukaryota,38HPW@33154|...",4891|Saccharomycetes,S,von Willebrand factor (vWF) type A domain,-,-,-,-,-,-,-,-,-,-,-,-,-
YALI0_A00418g,4952.CAG83536,0.000000e+00,1081.0,"KOG2504@1|root,KOG2504@2759|Eukaryota,39S9J@33...",4890|Ascomycota,G,to uniprot Q08268 Saccharomyces cerevisiae YOL...,-,-,-,-,-,-,-,-,-,-,-,-,MFS_1
YALI0_A00440g,4952.CAG83537,4.670000e-116,333.0,"2CUXP@1|root,2S4F4@2759|Eukaryota,3A7DW@33154|...",4891|Saccharomycetes,S,Saccharomyces cerevisiae,-,-,-,-,-,-,-,-,-,-,-,-,FUN14


In [11]:
DF_kegg = DF_eggnog[['KEGG_ko','KEGG_Pathway','KEGG_Module','KEGG_Reaction']]

# Melt dataframe
DF_kegg = DF_kegg.reset_index().melt(id_vars='locus_tag')

# Remove null values
DF_kegg = DF_kegg[DF_kegg.value.notnull()]

# Split comma-separated values into their own rows
list2struct = []
for name,row in DF_kegg.iterrows():
    for val in row.value.split(','):
        list2struct.append([row.locus_tag,row.variable,val])

DF_kegg = pd.DataFrame(list2struct,columns=['gene_id','database','kegg_id'])

# Remove ko entries, as only map entries are searchable in KEGG pathway
DF_kegg = DF_kegg[~DF_kegg.kegg_id.str.startswith('ko')]

DF_kegg.head()

,gene_id,database,kegg_id
2,YALI0_A00396g,KEGG_ko,-
3,YALI0_A00418g,KEGG_ko,-
4,YALI0_A00440g,KEGG_ko,-
6,YALI0_A00528g,KEGG_ko,-
10,YALI0_A00781g,KEGG_ko,-


In [12]:
DF_annot

,accession,start,end,strand,gene_name,old_locus_tag,gene_product,ncbi_protein
locus_tag,,,,,,,,
YALI0_B00110g,NC_006068.1,234.0,764.0,+,None,YALI0B00110g,YALI0B00110p,XP_500334.1
YALI0_E00110g,NC_006071.1,435.0,1553.0,-,None,YALI0E00110g,YALI0E00110p,XP_503359.1
YALI0_B00132g,NC_006068.1,1930.0,3360.0,-,None,YALI0B00132g,YALI0B00132p,XP_500335.1
YALI0_E00132g,NC_006071.1,2299.0,3738.0,-,None,YALI0E00132g,YALI0E00132p,XP_503360.1
YALI0_D00110g,NC_006070.1,3558.0,4058.0,-,None,YALI0D00110g,YALI0D00110p,XP_502226.1
...,...,...,...,...,...,...,...,...
YALI0_E35046g,NC_006071.1,4188366.0,4190297.0,-,None,YALI0E35046g,YALI0E35046p,XP_504797.2
YALI0_E35112g,NC_006071.1,4193407.0,4195506.0,-,None,YALI0E35112g,YALI0E35112p,XP_504799.1
YALI0_E35156g,NC_006071.1,4201039.0,4201245.0,+,None,YALI0E35156g,YALI0E35156p,XP_504800.2


In [16]:
DF_eggnog.COG_category

locus_tag
YALI0_A00264g    J
YALI0_A00286g    T
YALI0_A00396g    S
YALI0_A00418g    G
YALI0_A00440g    S
                ..
YALI0_F15631g    E
YALI0_F16379g    O
YALI0_F16819g    C
YALI0_F16852g    T
YalifMp16        T
Name: COG_category, Length: 490, dtype: object

In [18]:
DF_annot['COG'] = DF_eggnog.COG_category

# Make sure COG only has one entry per gene
DF_annot['COG'] = [item[0] if isinstance(item,str) else item for item in DF_annot['COG']]

In [24]:
cog_list = []

for gene in DF_annot.index:
     cog_list.append(DF_eggnog.loc[gene]['COG_category'])
        
DF_annot['COG'] = cog_list

# Make sure COG only has one entry per gene
DF_annot['COG'] = [item[0] if isinstance(item,str) else item for item in DF_annot['COG']]

KeyError: 'YALI0_B00110g'

In [27]:
DF_annot

,accession,start,end,strand,gene_name,old_locus_tag,gene_product,ncbi_protein,COG
locus_tag,,,,,,,,,
YALI0_B00110g,NC_006068.1,234.0,764.0,+,None,YALI0B00110g,YALI0B00110p,XP_500334.1,NaN
YALI0_E00110g,NC_006071.1,435.0,1553.0,-,None,YALI0E00110g,YALI0E00110p,XP_503359.1,NaN
YALI0_B00132g,NC_006068.1,1930.0,3360.0,-,None,YALI0B00132g,YALI0B00132p,XP_500335.1,NaN
YALI0_E00132g,NC_006071.1,2299.0,3738.0,-,None,YALI0E00132g,YALI0E00132p,XP_503360.1,NaN
YALI0_D00110g,NC_006070.1,3558.0,4058.0,-,None,YALI0D00110g,YALI0D00110p,XP_502226.1,NaN
...,...,...,...,...,...,...,...,...,...
YALI0_E35046g,NC_006071.1,4188366.0,4190297.0,-,None,YALI0E35046g,YALI0E35046p,XP_504797.2,NaN
YALI0_E35112g,NC_006071.1,4193407.0,4195506.0,-,None,YALI0E35112g,YALI0E35112p,XP_504799.1,NaN
YALI0_E35156g,NC_006071.1,4201039.0,4201245.0,+,None,YALI0E35156g,YALI0E35156p,XP_504800.2,NaN


In [32]:
DF_eggnog

,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,EC,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
locus_tag,,,,,,,,,,,,,,,,,,,,
YALI0_A00264g,4952.CAG83529,0.000000e+00,2194.0,"COG0060@1|root,KOG0434@2759|Eukaryota,38CIS@33...",4891|Saccharomycetes,J,Belongs to the class-I aminoacyl-tRNA syntheta...,ILS1,"GO:0003674,GO:0003824,GO:0004812,GO:0004822,GO...",6.1.1.5,ko:K01870,"ko00970,map00970","M00359,M00360",R03656,"RC00055,RC00523","ko00000,ko00001,ko00002,ko01000,ko01007,ko03016",-,-,-,"Anticodon_1,tRNA-synt_1"
YALI0_A00286g,4952.CAG83530,3.570000e-260,713.0,"COG1163@1|root,KOG1487@2759|Eukaryota,38DRY@33...",4891|Saccharomycetes,T,to Saccharomyces cerevisiae RBG1 (YAL036C),-,"GO:0000166,GO:0001882,GO:0001883,GO:0002181,GO...",-,ko:K06944,-,-,-,-,ko00000,-,-,-,"MMR_HSR1,MMR_HSR1_Xtn,TGS"
YALI0_A00396g,4952.CAG83535,7.670000e-243,681.0,"28MZB@1|root,2QUI6@2759|Eukaryota,38HPW@33154|...",4891|Saccharomycetes,S,von Willebrand factor (vWF) type A domain,-,-,-,-,-,-,-,-,-,-,-,-,-
YALI0_A00418g,4952.CAG83536,0.000000e+00,1081.0,"KOG2504@1|root,KOG2504@2759|Eukaryota,39S9J@33...",4890|Ascomycota,G,to uniprot Q08268 Saccharomyces cerevisiae YOL...,-,-,-,-,-,-,-,-,-,-,-,-,MFS_1
YALI0_A00440g,4952.CAG83537,4.670000e-116,333.0,"2CUXP@1|root,2S4F4@2759|Eukaryota,3A7DW@33154|...",4891|Saccharomycetes,S,Saccharomyces cerevisiae,-,-,-,-,-,-,-,-,-,-,-,-,FUN14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YALI0_F15631g,4952.CAG84185,1.930000e-103,313.0,"COG0531@1|root,KOG1289@2759|Eukaryota,38FDM@33...",4891|Saccharomycetes,E,GABA transport protein involved in the utiliza...,UGA4,"GO:0000322,GO:0000323,GO:0000324,GO:0000329,GO...",-,-,-,-,-,-,-,-,-,"iMM904.YDL210W,iND750.YDL210W",AA_permease_2
YALI0_F16379g,4952.CAG84221,0.000000e+00,1154.0,"COG5021@1|root,KOG0942@2759|Eukaryota,38D0N@33...",4891|Saccharomycetes,O,to Saccharomyces cerevisiae HUL5 (YGL141W),HUL5,"GO:0000209,GO:0000502,GO:0003674,GO:0003824,GO...",2.3.2.26,ko:K10589,"ko04120,map04120",-,-,-,"ko00000,ko00001,ko01000,ko04121",-,-,-,HECT
YALI0_F16819g,4952.CAG84248,1.200000e-174,490.0,"COG4266@1|root,KOG0769@2759|Eukaryota,38FEK@33...",4891|Saccharomycetes,C,Belongs to the mitochondrial carrier (TC 2.A.2...,-,-,-,ko:K13354,"ko04146,map04146",-,-,-,"ko00000,ko00001,ko02000",2.A.29.20.1,-,-,Mito_carr
